# Segmenting & Clustering Neighborhoods in Toronto, Canada

### Applied Data Science Capstone (Coursera) - Week 3 Assignment  

### Author: Patrick de Guzman

# Introduction
The following will be a report for the Week 3 Assignment in the Applied Data Science Capstone course offered through Coursera. The Foursquare API will be used to explore neighborhoods in the city of Toronto, Canada. More specifically, we will use the Fouresquare API to identify types of venues within Toronto neighbourhoods (which we will scrape from the web), and afterwards, k-means clustering will be used to group neighborhoods based on the types of venues within them to visually map the city's 'character'. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Preprocess the Dataset</a>

2. <a href="#item2">Obtaining Neighborhood Coordinates (Lat, Lon)</a>

3. <a href="#item3">Exploratory Analysis and Clustering</a>  
    a. <a href="#sub1">Map of Toronto Neighborhoods</a>  
    b. <a href="#sub2">Top Venues by Neighborhood</a>  
    c. <a href="#sub3">K-Means Clustering: Neighborhood Cluster Map</a>  
    d. <a href="#sub4">Examining Clusters</a>     
</font>
</div>

##### Required Dependencies


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from bs4 import BeautifulSoup # import BeautifulSoup for web scraping
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


<a id = 'item1'></a>

## 1. Download and Preprocess the Dataset <a href="item1"></a>

In [2]:
# Set up url of wikipedia page to extract table of Toronto postal codes
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc, 'lxml')

# From BeautifulSoup object, extract table
my_table = soup.find('table',{'class':'wikitable sortable'})

df = pd.read_html(str(my_table))[0]
df.Borough.replace('Not assigned', np.nan, inplace = True)
df = df.dropna(axis = 0) 
df.reset_index(drop = True, inplace = True)
df.columns = ['PostalCode', 'Borough','Neighborhood']
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Additional cleaning will be performed to do the following: 
1. Assign Borough names to Neighborhood names where Borough is present but Neighborhood is 'Not assigned', and
2. Combine neighbourhoods together in the event of duplicate postal codes

First, we assign the borough names to neighborhoods with missing values: 

In [3]:
df['Neighborhood'].replace('Not assigned', df['Borough'], inplace = True)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Next, we replace the 'Neighborhood' column with a concatenated version of all neighborhoods within each unique 'PostalCode'. To do this, we can create a for loop to cycle through each unique postal code, subset the original data frame based on each postal code, and within this subset, create a concatenated string of all resulting neighborhood values. 

The resulting final listing will be a data frame of unique postal codes and boroughs with strings of several neighborhoods (for those boroughs that have several neighborhoods).

In [4]:
bor_list = []
nh_list = []

for pcode in df.PostalCode.unique(): 
    subset = df[df.PostalCode == pcode] # subset original df by postal code
    
    neighborhoods = "" # initialize empty borough string
    for neighborhood in subset.Neighborhood:
        neighborhoods += str(neighborhood) + ", "
    
    neighborhoods = neighborhoods.rstrip()[:-1] # remove trailing spaces + ending comma

    bor_list.append(subset.Borough.iloc[0])
    nh_list.append(neighborhoods)


df_final = pd.DataFrame(data = {'PostalCode':df.PostalCode.unique(), 'Borough': bor_list, 'Neighborhoods': nh_list})

df_final = df_final.sort_values(by = ['PostalCode'])
df_final.reset_index(drop = True, inplace = True) 
df_final

,PostalCode,Borough,Neighborhoods
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<a id='item2'></a>

## 2. Obtaining Neighborhood Coordinates (Lat, Lon)

Now that we have the preprocessed dataframe, we will obtain the coordinates for each neighborhood in the dataframe using the geocoder package. However, from running the loop below, the package doesn't appear to be working reliably as the loop runs indefinitely when executed. Therefore, the below code is shown only for demonstration purposes. 

In [5]:
# import geocoder

# lat_lng_list = None

# pcodes = df_final.PostalCode.unique()

# for code in pcodes:
#     lat_lng_coords = None
#     while(lat_lng_coords is None): 
#         g = geocoder.google('{}, Toronto, Ontario'.format(code))
#         lat_lng_coords = g.latlng
#     lat_lng_list.append(lat_lng_coords)
    
# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

As a result, we will use the coordinate data provided from the link [here](https://cocl.us/Geospatial_data):

In [6]:
coordinates_url = 'https://cocl.us/Geospatial_data'
coordinates = pd.read_csv(coordinates_url)

df_coordinates = pd.merge(left = df_final, right = coordinates, left_on = 'PostalCode', right_on = 'Postal Code')
df_coordinates.drop('Postal Code', axis = 1, inplace = True)
df_coordinates

,PostalCode,Borough,Neighborhoods,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<a id='item3'></a>

## 3. Exploratory Analysis and Clustering

In [1]:
{
    "tags": [
        "remove_input",
    ]
}
CLIENT_ID = 'P02OB2VFEUPYSEAGURXEOSFE2UVRYFRSZOW1CWA1XSEG0LIM' 
CLIENT_SECRET = 'LVEBZNC05SC2BF4140E5J3N1VJB5QHIGMWZNRDJAE424A31A' 
VERSION = '20180605' 

<a id='sub1'></a>

### Map of Toronto Neighborhoods

First, we can create an initial map of neighborhoods in Toronto. For the purposes of this analysis, we will subset the data to analyze only the boroughs with 'Toronto' in the name:

In [8]:
# Subset data
index = df_coordinates['Borough'].str.contains('Toronto')
df_subset = df_coordinates[index]

# Obtain coordinates of Toronto, Canada
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [9]:
# Create map of Toronto, ON 
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# Add neighbourhood markers to map
for lat, lng, borough, neighborhood in zip(df_subset['Latitude'], df_subset['Longitude'], df_subset['Borough'], df_subset['Neighborhoods']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True, 
        fill_color = 'blue',
        fill_opacity = 0.8,
        parse_html = False).add_to(map_toronto)

map_toronto

<a id='sub2'></a>

### Top Venues by Neighborhood

Using the Foursquare API, we can explore the venues in each borough with the following function:

In [10]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names = df_subset['Neighborhoods'], 
                                 latitudes = df_subset['Latitude'],
                                 longitudes = df_subset['Longitude']
                                )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [12]:
print(toronto_venues.shape)
toronto_venues.head()

(943, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
2,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
3,The Beaches,43.676357,-79.293031,Balmy Beach Playground,43.676078,-79.290805,Playground
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


We can review the number of venues returned for each neighborhood using the following: 

In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",61,61,61,61,61,61
Berczy Park,6,6,6,6,6,6
"Brockton, Exhibition Place, Parkdale Village",16,16,16,16,16,16
Business Reply Mail Processing Centre 969 Eastern,7,7,7,7,7,7
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",5,5,5,5,5,5
"Cabbagetown, St. James Town",20,20,20,20,20,20
Central Bay Street,31,31,31,31,31,31
"Chinatown, Grange Park, Kensington Market",74,74,74,74,74,74
Christie,6,6,6,6,6,6


We can also identify the number of unique categories from all of the returned venues: 

In [14]:
print('There are {} unique categories of venues.'.format(len(toronto_venues['Venue Category'].unique())))

There are 191 unique categories of venues.


In order to begin clustering the neighbourhoods based on similar venue types, we need to convert the occurrence of each type of venue into dummy variables for each neighborhood (i.e., one-hot encoding):

In [15]:
# One-hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")

# Add neighborhood column back to dataframe and move to first column
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

fixed_columns = [['Neighborhood'] + list(toronto_onehot.columns[toronto_onehot.columns != 'Neighborhood'])]
toronto_onehot = toronto_onehot[fixed_columns[0]]

print(toronto_onehot.shape)
toronto_onehot.head(10)


(943, 191)


,Neighborhood,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

To examine each neighborhood, we must group them by the 'Neighborhood' column and use the aggregate function mean of the frequency of occurrence for each venue category: 


In [16]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(36, 191)


,Neighborhood,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0,0.0,0.0,0.0,0.049180,0.000000,0.000000,0.049180,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.049180,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00000,0.016393,0.000000,0.000000,0.016393,0.016393,0.000000,0.000000,0.00,0.032787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065574,0.00,0.000000,0.016393,0.000000,0.000000,0.016393,0.0,0.000000,0.016393,0.000000,0.00000,0.000000,0.016393,0.016393,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.016393,0.000000,0.000000,0.016393,0.000000,0.000000,0.016393,0.000000,0.000000,0.016393,0.0,0.000000,0.016393,0.000000,0.000000,0.000000,0.049180,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.016393,0.000000,0.000000,0.00,0.016393,0.000000,0.0000,0.016393,0.000000,0.032787,0.00,0.016393,0.000000,0.000000,0.000000

Let's also analyze the top 5 most common types of venues per each neighborhood: 


In [17]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['VENUE','FREQ']
    temp = temp.iloc[1:]
    temp['FREQ'] = temp['FREQ'].astype(float)
    temp = temp.round({'FREQ': 2})
    print(temp.sort_values('FREQ', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 VENUE  FREQ
0           Steakhouse  0.07
1          Coffee Shop  0.07
2                  Bar  0.05
3  American Restaurant  0.05
4                Hotel  0.05


----Berczy Park----
                VENUE  FREQ
0      Breakfast Spot  0.17
1        Concert Hall  0.17
2          Steakhouse  0.17
3        Liquor Store  0.17
4  Italian Restaurant  0.17


----Brockton, Exhibition Place, Parkdale Village----
                 VENUE  FREQ
0       Sandwich Place  0.12
1                 Café  0.12
2  Sporting Goods Shop  0.06
3  Japanese Restaurant  0.06
4                 Park  0.06


----Business Reply Mail Processing Centre 969 Eastern----
                  VENUE  FREQ
0                  Park  0.14
1  Fast Food Restaurant  0.14
2    Light Rail Station  0.14
3        Farmers Market  0.14
4         Auto Workshop  0.14


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                V

In order to place this data into a pandas dataframe, we will use the following functions to return the sorted venues in descending order, as well as to create the new dataframe and display the top 10 venues within each neighborhood: 

In [18]:
## Function to sort the venues 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Set up the dataframe parameters 
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Asian Restaurant,Bar,Hotel,American Restaurant,Pizza Place,Thai Restaurant,Japanese Restaurant,Seafood Restaurant
1,Berczy Park,Liquor Store,Italian Restaurant,Breakfast Spot,Steakhouse,Beer Bar,Concert Hall,General Travel,General Entertainment,Fast Food Restaurant,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Café,Sandwich Place,Pet Store,Sporting Goods Shop,Japanese Restaurant,Falafel Restaurant,Park,Grocery Store,Coffee Shop,Caribbean Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Brewery,Park,Light Rail Station,Fast Food Restaurant,Farmers Market,Auto Workshop,Garden,Dog Run,Festival,Falafel Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,Coffee Shop,Yoga Studio,Dog Run,Fish & Chips Shop,Festival,Fast Food Restaurant


<a id='sub3'></a>

### K-Means Clustering: Toronto Neighborhood Cluster Map
Finally, we can use the created dataframe of Toronto neighborhoods to cluster the neighborhoods into similar groupings based on their top venues. 

In [19]:
kclusters = 16

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# Run k-means Clustering 
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

kmeans.labels_

array([ 0, 11,  8,  4, 13,  8, 15,  8, 12,  0,  8,  8,  7, 10,  8,  0,  8,
        8,  8,  8,  2,  0,  6,  1,  9,  5,  3,  0,  8,  8,  8,  8,  8, 14,
        0,  0])

In [20]:
# add clustering labels
if neighborhoods_venues_sorted.columns.contains('Cluster Labels'):
    neighborhoods_venues_sorted.drop(['Cluster Labels'], axis = 'columns', inplace = True)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_subset

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = pd.merge(left = toronto_merged, right = neighborhoods_venues_sorted.set_index('Neighborhood'), left_on='Neighborhoods', right_on='Neighborhood')

toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,14,Park,Other Great Outdoors,Playground,Trail,Diner,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Drugstore
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Ice Cream Shop,Sports Bar,Indian Restaurant,Fruit & Vegetable Store,Italian Restaurant,Juice Bar,Diner,Dessert Shop,Cosmetics Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Burger Joint,Movie Theater,Sushi Restaurant,Board Shop,Italian Restaurant,Burrito Place,Pub,Fish & Chips Shop,Ice Cream Shop,Hotel
3,M4M,East Toronto,Studio District,43.659526,-79.340923,8,Coffee Shop,Café,Italian Restaurant,Comfort Food Restaurant,Bank,Convenience Store,Diner,Clothing Store,Chinese Restaurant,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Photography Studio,Lawyer,Jewelry Store,Dive Bar,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Drugstore
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,7,Breakfast Spot,Gym,Grocery Store,Dive Bar,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Boutique,Concert Hall,Fish & Chips Shop,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Drugstore
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,8,Dessert Shop,Coffee Shop,Café,Thai Restaurant,Pizza Place,Italian Restaurant,Seafood Restaurant,New American Restaurant,Sushi Restaurant,Salon / Barbershop
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,6,Construction & Landscaping,Park,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,10,Coffee Shop,Athletics & Sports,Liquor Store,Supermarket,Dog Run,Fish & Chips Shop,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant


From the above dataframe, we've clustered each neighborhood (see the 'Cluster labels' column) based on the similarity of venue types. Next, we can visualize the clusters in a folium map: 

In [21]:
# Create map 
map_clusters = folium.Map(location = [latitude,longitude], zoom_start = 12)

# Set color scheme for clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to cluster map 
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhoods'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat,lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True, 
        fill_color = rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

From the final neighborhood clustering map above, a large k-means cluster 'n' of 16 was chosen to see more granularity and separation between neighborhoods (k-means cluster 'n' of 5 yielded overly-simplistic view of Toronto's diverse neighborhoods, simply grouping nearly all of Downtown Toronto together into a single cluster). 

 

<a id = 'sub4'></a>

### Examining Clusters
From the final cluster map, we can make inferences about some of the major clusters of neighbourhoods by looking at each major cluster in more detail.

##### Cluster no. 8: Downtown Core 

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,8,Coffee Shop,Café,Italian Restaurant,Comfort Food Restaurant,Bank,Convenience Store,Diner,Clothing Store,Chinese Restaurant,Cheese Shop
7,Central Toronto,8,Dessert Shop,Coffee Shop,Café,Thai Restaurant,Pizza Place,Italian Restaurant,Seafood Restaurant,New American Restaurant,Sushi Restaurant,Salon / Barbershop
11,Downtown Toronto,8,Pizza Place,Café,Coffee Shop,Jewelry Store,Dive Bar,Restaurant,Bakery,Indian Restaurant,Diner,Italian Restaurant
13,Downtown Toronto,8,Gym / Fitness Center,History Museum,Bakery,Furniture / Home Store,Food Truck,Light Rail Station,Park,Coffee Shop,Bus Stop,Breakfast Spot
14,Downtown Toronto,8,Coffee Shop,Middle Eastern Restaurant,Café,Sandwich Place,Movie Theater,Hotel,Clothing Store,Tea Room,Beer Bar,Fast Food Restaurant
15,Downtown Toronto,8,Gastropub,Coffee Shop,Restaurant,Italian Restaurant,Japanese Restaurant,BBQ Joint,Hotel,Speakeasy,Breakfast Spot,Creperie
19,Downtown Toronto,8,Coffee Shop,Café,Sandwich Place,Pizza Place,Park,Italian Restaurant,Hotel,Sports Bar,Sporting Goods Shop,Boat or Ferry
20,Downtown Toronto,8,Coffee Shop,Restaurant,Deli / Bodega,Café,American Restaurant,Bakery,Thai Restaurant,Bar,Salad Place,Gym
21,Downtown Toronto,8,Coffee Shop,Café,Restaurant,Deli / Bodega,Hotel,Gastropub,Burger Joint,Gym,Salad Place,American Restaurant
23,Central Toronto,8,Park,Sandwich Place,Liquor Store,Coffee Shop,Burger Joint,Donut Shop,Café,Middle Eastern Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant


From above, it appears that Toronto's downtown core (spilling into a few neighbourhoods in both the East and West side of the core) is characterized mostly by cafes/coffee shops,  bars/pubs, and a large variety of restaurants (Italian, Asian, Indan, American, Chinese).

##### Cluster no. 0: Little Portugal, Adelaide, The Beaches, Dovercourt, Dufferin, Danforth

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Ice Cream Shop,Sports Bar,Indian Restaurant,Fruit & Vegetable Store,Italian Restaurant,Juice Bar,Diner,Dessert Shop,Cosmetics Shop
2,East Toronto,0,Burger Joint,Movie Theater,Sushi Restaurant,Board Shop,Italian Restaurant,Burrito Place,Pub,Fish & Chips Shop,Ice Cream Shop,Hotel
12,Downtown Toronto,0,Gay Bar,Burger Joint,Japanese Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Salon / Barbershop,Bubble Tea Shop,Restaurant,Ramen Restaurant
18,Downtown Toronto,0,Coffee Shop,Steakhouse,Asian Restaurant,Bar,Hotel,American Restaurant,Pizza Place,Thai Restaurant,Japanese Restaurant,Seafood Restaurant
30,West Toronto,0,Gym / Fitness Center,Music Venue,Supermarket,Middle Eastern Restaurant,Bar,Bank,Bakery,Pizza Place,Pharmacy,Cupcake Shop
31,West Toronto,0,Bar,Asian Restaurant,New American Restaurant,Wine Bar,Vietnamese Restaurant,Coffee Shop,Bakery,Pizza Place,Record Shop,Boutique
34,West Toronto,0,Sushi Restaurant,French Restaurant,Bakery,Sandwich Place,Falafel Restaurant,Smoothie Shop,Café,Fish & Chips Shop,Food,Food & Drink Shop


From above, cluster no. 0 is mostly characterized by Asian (Sushi, Chinese, Japanese, Asian, Thai) and Italian (Pizza, Italian) restaurants, as well as coffee shops/cafes/bakeries. 